In [38]:
import pandas as pd
import numpy as np
import random
import os
import keras
from keras import models
from keras.models import load_model
from keras import layers
from keras.utils import to_categorical
from keras.models import Sequential
from keras.layers import Dense
from keras.wrappers.scikit_learn import KerasRegressor
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import KFold
from sklearn.preprocessing import StandardScaler
from sklearn.pipeline import Pipeline
from scipy.ndimage import rotate
np.random.seed(42)

In [18]:
dd = pd.read_csv("train.csv")
dd.head()

M = int(0.8 * len(dd))

dd2 = dd[['x', 'y', 'z', 'Vx', 'Vy', 'Vz', 'x_sim', 'y_sim', 'z_sim', 'Vx_sim', 'Vy_sim', 'Vz_sim']]
dd2 = dd2.sample(frac=1).reset_index(drop=True)

dd2.head()

train_input = dd2.iloc[0:M][['x_sim', 'y_sim', 'z_sim', 'Vx_sim', 'Vy_sim', 'Vz_sim']].values
train_output = dd2.iloc[0:M][['x', 'y', 'z', 'Vx', 'Vy', 'Vz']].values

test_input = dd2.iloc[M:][['x_sim', 'y_sim', 'z_sim', 'Vx_sim', 'Vy_sim', 'Vz_sim']].values
test_output = dd2.iloc[M:][['x', 'y', 'z', 'Vx', 'Vy', 'Vz']].values
# Acceso a fila
# dd.iloc[i]

# Accesso a columna
# dd['name']
# dd.name

In [11]:
def create_model():
    model = Sequential()
    model.add(Dense(12, input_dim = 6, kernel_initializer='normal', activation='relu'))
    model.add(Dense(6, kernel_initializer='normal'))
    model.compile(loss = 'mean_squared_error', optimizer='adam')
    return model

In [12]:
model = create_model()
model.summary()
history = model.fit(train_input, train_output, epochs=3, batch_size=32)
model.save('model2.h5') 

Model: "sequential_4"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_9 (Dense)              (None, 12)                84        
_________________________________________________________________
dense_10 (Dense)             (None, 6)                 78        
Total params: 162
Trainable params: 162
Non-trainable params: 0
_________________________________________________________________
Epoch 1/3
519929/519929 [==============================] - 22s 43us/step - loss: 27443496.2856
Epoch 2/3
519929/519929 [==============================] - 22s 43us/step - loss: 22840682.1771
Epoch 3/3
519929/519929 [==============================] - 21s 41us/step - loss: 22822532.8771


In [39]:
model = load_model('model1.h5')
results = model.evaluate(test_input, test_output)
results

129983/129983 [==============================] - 2s 19us/step


22174979.498026665

In [40]:
error_train = np.mean((train_input-train_output)**2)
error_test = np.mean((test_input-test_output)**2)

print((error_test-results)/error_test * 100)

1.9347056255041315


In [109]:
type(ids)

pandas.core.series.Series

In [63]:
FINAL_TEST_RAW = pd.read_csv("test.csv")
FINAL_TEST = FINAL_TEST_RAW[['x_sim', 'y_sim', 'z_sim', 'Vx_sim', 'Vy_sim', 'Vz_sim']].values
ids = FINAL_TEST_RAW['id']

RESULTS = model.predict(FINAL_TEST)

In [ ]:
df = pd.DataFrame(data=RESULTS,
          index=np.array(range(1, 1+len(RESULTS))),
          columns=['x', 'y', 'z', 'Vx', 'Vy', 'Vz'])

df.insert(0, 'id', 0)




df['id'] = ids2

#df.rename(columns={'x_sim' : 'x', 'y_sim' : 'y', 'z_sim' : 'z', 'Vx_sim' : 'Vx', 'Vy_sim' : 'Vy', 'Vz_sim' : 'Vz'}, inplace=True)

df['id'] = pd.Series(df['id'], dtype='int32')



0       1
0    3927
1    3928
2    3929
3    3930
dtype: int64

In [122]:
#df['id'] = df['id'].astype(int)